In [11]:
import pandas as pd 


def load_unitig_data(path) -> pd.DataFrame:
    # columns correspond to unitigs so we must transpose this table
    sr = pd.read_csv(path, sep=' ')
    sr.set_index('pattern_id',inplace=True)
    return sr.T

azm_sr = load_unitig_data("azm_sr_gwas_filtered_unitigs.Rtab")
cfx_sr = load_unitig_data("cfx_sr_gwas_filtered_unitigs.Rtab")
cip_sr = load_unitig_data("cip_sr_gwas_filtered_unitigs.Rtab")


metadata = pd.read_csv('metadata.csv')

In [3]:
print(metadata.head(1))

    Sample_ID    Year Country Continent Beta.lactamase Azithromycin  \
0  ERR1549286  2015.0      UK    Europe            NaN         >256   

  Ciprofloxacin Ceftriaxone Cefixime Tetracycline  ... log2_cro_mic  \
0           NaN       0.016      NaN          NaN  ...    -5.965784   

  log2_cfx_mic  log2_tet_mic  log2_pen_mic  azm_sr  cip_sr  cro_sr  cfx_sr  \
0          NaN           NaN           NaN     1.0     NaN     0.0     NaN   

   tet_sr  pen_sr  
0     NaN     NaN  

[1 rows x 31 columns]
09_008


In [19]:

samples = azm_sr.index

# some random unitig from azm_sr
azm_unitig = azm_sr['CCCCACCGAAATCAAAGGCAAATATGTTCAGAAA']
j = 0
for i in range(len(azm_unitig)):
    if azm_unitig[samples[i]]:
        j += 1
        # the unitig is present in the sample
        print(samples[i])

print("present in ", j, " azm samples")

11791_8#93
11791_8#94
11791_8#95
11791_8#96
11792_5#3
11792_5#33
11792_5#35
11792_5#39
11792_5#4
11792_5#5
12273_1#10
12273_1#12
12273_1#13
12273_1#47
12273_1#55
SRR1661157
SRR1661194
SRR1661263
SRR1661295
SRR1661316
SRR1661320
SRR1661323
SRR1661153
SRR2736218
SRR2736289
SRR1661322
16853_4#29
17138_1#2
17150_2#88
17328_3#36
17427_1#11
17428_7#22
17428_7#32
17428_7#41
17428_7#42
17428_7#43
SRR3357229
10356_1#10
10356_1#41
10356_1#62
10356_1#85
10625_6#23
10625_6#50
10868_8#7
10900_8#49
10900_8#70
10900_8#74
11792_4#61
11792_4#69
8289_2#12
8289_2#14
9716_3#45
8727_8#4
8289_2#11
8727_8#76
17176_1#69
17176_1#36
15335_2#87
15335_4#6
17150_8#45
17176_1#64
17176_1#65
17176_1#67
15335_7#7
15335_3#90
15335_3#67
17150_8#66
17176_1#58
17176_1#60
15335_3#13
15335_6#48
17225_3#33
17176_1#23
17176_1#87
15335_5#92
15335_5#93
15335_5#94
17150_8#6
17176_1#77
17176_1#79
15335_6#69
17150_8#2
15335_6#23
16043_2#21
15335_6#2
15335_5#66
15335_4#69
15335_5#67
15335_5#68
15335_2#39
SRR5827361
SRR5827025
SRR58